In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn
import scipy.optimize as op
seaborn.set(font_scale=1.5,style='ticks')

/Users/yves/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Run with jax

In [19]:
dim = 5
x0 = np.random.normal(size=(dim,1))
A = np.random.normal(size=(dim,dim))

x = np.array([np.linalg.matrix_power(A,i)@x0 + np.random.normal(size=(5,1)) for i in range(9)])

In [20]:
x[0]

array([[-0.95793199],
       [ 0.36729174],
       [ 2.9574978 ],
       [ 0.40501579],
       [ 1.4323647 ]])

In [21]:
x0

array([[ 0.04634898],
       [-0.40513895],
       [ 0.45600041],
       [-0.28774787],
       [ 0.56748995]])

In [22]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random


In [23]:
def predict(Ahat,x,y):
    Ahat = jnp.reshape(Ahat,(5,5))
    return jnp.sum((jnp.dot(jnp.linalg.matrix_power(Ahat,2),x)-y)**2)

In [26]:

grad_predict = grad(predict)

def grad_wrap(Ahat,x,y):
    Ahat = jnp.reshape(Ahat,(5,5))
    return np.array(grad_predict(Ahat,x,y)).flatten()

In [29]:
predict(A,x[0],y[1])

DeviceArray(53.46995, dtype=float32)

In [30]:
grad_wrap(A,x[0],y[1])

array([  1.7994652,  -4.0419493, -51.338753 ,   9.744552 , -18.75742  ,
       -15.890802 ,   2.949784 ,   6.131092 ,  16.56932  ,   8.69558  ,
         6.374323 ,  -8.022389 , -95.871475 ,  14.78779  , -36.269325 ,
        29.32734  ,  -4.4623246,   2.092739 , -33.656174 , -11.342876 ,
         6.6361294,  -5.3563666, -58.894913 ,   6.007004 , -23.400902 ],
      dtype=float32)

In [28]:
res = op.minimize(predict,
           Ahat.flatten(),
           (x,y),
           jac=grad_wrap,
           method='BFGS'
           )

ValueError: Incompatible shapes for broadcasting: ((5, 9, 1), (1, 5, 1))

In [615]:
res.x.reshape(5,5)

array([[-1.22460316,  0.72622814,  0.50708658,  0.54978612, -1.00648622],
       [ 1.82127266, -1.06875035,  1.47263374,  0.38960624,  1.29957188],
       [ 0.54046966,  0.7858748 ,  2.30273027,  0.88306787,  0.72985877],
       [-0.34367772, -1.15917902, -1.96031598, -0.9978303 , -1.41348504],
       [ 0.21099819, -0.84988989, -1.82750732, -1.73886696, -0.69337155]])

In [618]:
np.linalg.matrix_power(res.x.reshape(5,5),2)@x

array([[ 4.36784692],
       [-5.1965338 ],
       [ 2.36050634],
       [-2.71350226],
       [-2.28097974]])

In [619]:
y

array([[ 4.36784739],
       [-5.19653372],
       [ 2.36050631],
       [-2.71350261],
       [-2.28098008]])

In [616]:
A

array([[ 1.74450561,  0.05476965, -1.26433905,  1.48453462, -0.17402451],
       [-1.21648828,  0.36858357,  0.74671068, -0.84791573, -0.5981811 ],
       [-0.76415638, -1.08190932, -1.53729033, -0.45533816,  0.09004409],
       [-1.25482215,  0.13408206,  0.15270299,  0.02274773,  1.13193254],
       [ 0.15826034,  0.2141965 , -1.95663259,  2.23788712, -0.75780314]])